# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import pickle
nltk.download('punkt')
nltk.download('wordnet')

from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('disaster_messages', engine)
X = df['message']
y = df.iloc[:,4:]
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print(X.shape)
print(y.shape)


(26216,)
(26216, 36)


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    word_tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for token in word_tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
    
    return clean_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

In [7]:
#pipeline.fit(X_train, y_train)

In [8]:
#y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
category_names = list(y)
# for i in range(36):
#     print(category_names[i], '\n', classification_report(y_test.iloc[:,i], y_pred[:,i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None, 'steps': [('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))], 'text_pipeline': Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), prepr

In [11]:
parameters = {
    'clf__estimator__learning_rate': [.7,1],
    'clf__estimator__n_estimators': [50,100],
}

cv = GridSearchCV(pipeline, param_grid = parameters)

cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.7, 1], 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
y_pred = cv.predict(X_test)

In [14]:
cv.best_params_

{'clf__estimator__learning_rate': 0.7, 'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
for i in range(36):
    category = category_names[i]
    accuracy = accuracy_score(y_test.iloc[:,i], y_pred[:,i])
    precision = precision_score(y_test.iloc[:,i], y_pred[:,i], average='weighted')
    recall = recall_score(y_test.iloc[:,i], y_pred[:,i], average='weighted')
    print(category)
    print("\tAccuracy: %.4f\tPrecision: %.4f\t Recall: %.4f\n" % (accuracy, precision, recall))

related
	Accuracy: 0.7843	Precision: 0.7583	 Recall: 0.7843

request
	Accuracy: 0.8989	Precision: 0.8932	 Recall: 0.8989

offer
	Accuracy: 0.9950	Precision: 0.9920	 Recall: 0.9950

aid_related
	Accuracy: 0.7670	Precision: 0.7689	 Recall: 0.7670

medical_help
	Accuracy: 0.9325	Precision: 0.9206	 Recall: 0.9325

medical_products
	Accuracy: 0.9563	Precision: 0.9476	 Recall: 0.9563

search_and_rescue
	Accuracy: 0.9729	Precision: 0.9673	 Recall: 0.9729

security
	Accuracy: 0.9809	Precision: 0.9645	 Recall: 0.9809

military
	Accuracy: 0.9723	Precision: 0.9662	 Recall: 0.9723

child_alone
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

water
	Accuracy: 0.9666	Precision: 0.9649	 Recall: 0.9666

food
	Accuracy: 0.9502	Precision: 0.9479	 Recall: 0.9502

shelter
	Accuracy: 0.9470	Precision: 0.9424	 Recall: 0.9470

clothing
	Accuracy: 0.9889	Precision: 0.9876	 Recall: 0.9889

money
	Accuracy: 0.9792	Precision: 0.9742	 Recall: 0.9792

missing_people
	Accuracy: 0.9895	Precision: 0.9866	 Recall:

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
filename = 'pipeline.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

NameError: name 'pickle' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.